# Notas realizadas durante la extracción de información

Se ha extraído la información de http://plants.ensembl.org/index.html para maíz (zea mays).
Después se procedió a obtener especies cercanas desde http://plants.ensembl.org/prot_tree_stats.html
Las especies elegidas fueron:
- Sorghum bicolor (Sorghum bicolor strain BTx623)
- Setaria italica (Setaria italica strain Yugu1)
- Oryza sativa Indica Group (Oryza sativa Indica Group strain 93-11)
- Oryza glumipatula (Oryza glumipatula)
- Oryza brachyantha (Oryza brachyantha strain IRGC:101232)
- Triticum aestivum (Triticum aestivum strain Chinese spring)

El criterio utilizado para la selección fue que las dos primeras especies eran las más cercanas en el árbol, mientras las 4 restante se eligieron al azar de todo el siguiente nivel que estaba disponible.

La información extraída de los transcriptomas viene en formato comprimido. Voy a generar un método que me permita leer esta información.

In [1]:
#método para leer los primeros num transcriptomas de un archivo comprimido
import gzip

def leer_transcriptomas(archivo, num):
    transcriptoma = ""
    f = gzip.open(archivo, "rt")
    for linea in f:
        if linea.startswith(">"):
            num -= 1
            if transcriptoma != "":
                print("TRANSCRIPTOMA: " + transcriptoma + "\n")
                transcriptoma = ""
            if num < 0:
                return
            print("CABECERA: " + linea)
        else:
            transcriptoma += linea.rstrip("\n")

Ahora leeré el archivo con los RNA codificantes.

In [2]:
#leyendo los primeros transcriptomas del archivo de RNA codificantes
print("INICIANDO...\n")
leer_transcriptomas(".\\httpplants.ensembl.orgindex.html\\Zea mays (maíz)\\cdna\\Zea_mays.B73_RefGen_v4.cdna.all.fa.gz", 3)
print("FIN...")

INICIANDO...

CABECERA: >Zm00001d035916_T001 cdna chromosome:B73_RefGen_v4:6:60137069:60138570:1 gene:Zm00001d035916 gene_biotype:protein_coding transcript_biotype:protein_coding description: UMP/CMP kinase1

TRANSCRIPTOMA: TCAAGCTGGGGCATTCCCTCCTGGGAAGAAGATCACGGTGGTGTTTGTCATAGGTATGCCATGCCTACTGTACTGTGTTTTTATCGAGCGAATGTATATGATCATATTTTTTGCCTAGGCGGCCCAGGCAGTGGCAAGGGCACCCAGTGCTCCAAGATTGTCAGACACTTTGGATTCACCCACCTGAGCGCTGGCGACCTCCTGCGCCAACAAGTTCAGTCAGATACTGAACACGGTGCCATGATCAAGAACCTGATGCACGAAGGGAAGCTCGTGCCGTCCGACATCATCGTCAGGCTCCTGCTCACGGCCATGCTCCAGAGCGGAAACGACAGGTTCCTCGTCGACGGATTCCCTCGAAACGAGGAGAACCGCCGAGCGTACGAGAGCGTTATTGGCATTGAGCCTGAGTTGGTGCTGTTCATCGACTGCCCAAGGGAGGAGCTGGAGCGGCGCATCCTGCACCGCGATCAGGGAAGAGACGACGACAACGTCGACACTATAAGGAAGCGGTTTCAGGTTTTCCACGACTCCACTCTACCTGTTGTTCTGTACTACGACAGAATGGGGAAGGTCCGACGGGTAGATGGCGCCAAATCCGCTGATGCAGTCTTTGACGATGTGAAGGCTATATTTACTCAGCTACTGACCACTCAGGTCCACTCACTCACTCACATTTATCTTCCTTTTTTTTTTCCAATTGATTGTTCTCTCCTGATAAAGCCTTGAAAAAACATCGGCGCTCGTCCGTCTATCCTCTTTTTGTTTCACTATGT

Y de forma similar, leeré el archivo con los RNA no codificantes.

In [3]:
#Leyendo los primeros transcriptomas del archivo de RNA no codificantes
print("INICIANDO...\n")
leer_transcriptomas(".\\httpplants.ensembl.orgindex.html\\Zea mays (maíz)\\ncrna\\Zea_mays.B73_RefGen_v4.ncrna.fa.gz", 3)
print("FIN...\n")

INICIANDO...

CABECERA: >ENSRNA049459199-T1 ncrna chromosome:B73_RefGen_v4:5:192896513:192896673:1 gene:ENSRNA049459199 gene_biotype:snRNA transcript_biotype:snRNA gene_symbol:U1 description:U1 spliceosomal RNA

TRANSCRIPTOMA: ATACTTACCTGGACGGGGTCAACGGGCTATCAAGAAGGCCTGTGGCCTAGATCAGTGGCCTTCATTGCACTTTAGAGGGTGCGCTGGTTTACCATCTCTCAAAGTGGGAGAGTAAACGTCATAATTTGTGGTAGCGGGGTAAGCGTCCGCGTGGCCCCTGC

CABECERA: >ENSRNA049476460-T1 ncrna chromosome:B73_RefGen_v4:9:44366481:44366671:-1 gene:ENSRNA049476460 gene_biotype:snRNA transcript_biotype:snRNA gene_symbol:U2 description:U2 spliceosomal RNA

TRANSCRIPTOMA: ATACCTTTTTCGGTCTTTTGGCTAAGATCAAGTGTAGTATCTACTTTTATCAGTTTAATATCTGATATATGGACCATGTGTCTACTTTGATATTAAATTTATTTTTTATGGTGAGGGTCCACAGTAGTTTGCCACTGGACCCTTGTGTGATGCCTAACCATTGCACTGCTGGTAGGATATGGCATACCCTA

CABECERA: >ENSRNA049462071-T1 ncrna chromosome:B73_RefGen_v4:9:118644589:118644784:-1 gene:ENSRNA049462071 gene_biotype:snRNA transcript_biotype:snRNA gene_symbol:U2 description:U2 spliceosomal RNA

TRANSC

En el caso de los no codificantes, me interesa filtrar aquellos que son lincrna. En la cabecera exite un marcador llamado gene_biotype (también sirve la cabecera transcript_biotype) que permite filtrarlos cuando tienen el valor lincRNA.
Generaré un segundo método para leer estos casos, aprovechando para filtrar aquellos con longitud mayor o igual a 200 nucleótidos.

In [4]:
#método para leer los primeros num lincrna de un archivo comprimido
import gzip

def es_lincrna(cabecera):
    for info in cabecera.split(" "):
        if info.split(":")[0] == "gene_biotype":
            if info.split(":")[1] == "lincRNA":
                return True
    return False

def leer_transcriptomas_lincrna(archivo, num):
    cabecera = ""
    transcriptoma = ""
    f = gzip.open(archivo, "rt")
    for linea in f:
        if linea.startswith(">"):
            if transcriptoma != "":
                if es_lincrna(cabecera) and (len(transcriptoma) >= 200):
                    num -= 1
                    print("CABECERA: " + cabecera + "\nTRANSCRIPTOMA: " + transcriptoma + "\n")
                transcriptoma = ""
            if num < 0:
                return
            cabecera = linea
        else:
            transcriptoma += linea.rstrip("\n")

Con el método definido, procederé a leer lincrna del archivo de RNA no codificantes.

In [5]:
#Leyendo los primeros lincRNA del archivo de RNA no codificantes
print("INICIANDO...\n")
leer_transcriptomas_lincrna(".\\httpplants.ensembl.orgindex.html\\Zea mays (maíz)\\ncrna\\Zea_mays.B73_RefGen_v4.ncrna.fa.gz", 3)
print("FIN...\n")

INICIANDO...

CABECERA: >Zm00001d001446_T001 ncrna chromosome:B73_RefGen_v4:6:151602192:151603028:1 gene:Zm00001d001446 gene_biotype:lincRNA transcript_biotype:lincRNA

TRANSCRIPTOMA: GACGGCGGGTTAATTTTATTATCCTCTGAGGTTTCTTTTGTAAAACAGCTTCGGCGAAGGGGTACGGGCTGATTTGCGTCGTTGGATCTCCATTCGACAGCTCAGATTAGATCCGAAACCTAAACGAACCGGTACGCGTTGAGGGTCGTCGATCAAGCAATCTACGGTCACGCTTTAATGCGTGAAGGGGTACGCAGCGTTTTAATCCCAAGCGTCCAATCTAGCATCGAACGGTCGCGGCTTAGTGACTCAGGGCTCTCATCATGAGTCATGACTGTCCATCCATGAATGGACGGCGCGGGGGCTTCTCCCACCTCCGAACCAAACAGCGGCGGCGCCAAACCAAACAACGACGTGCGCCATGGCCGCAGAACAACAAACTTGTACCCACTTGCCCAACACTTACACCGGATGTGCACTACATGTAGCGGGGCAAAGACGAACCAGGGTGGCTTGTGATTACCAGGACCCAAAGCGCCGCAGTTGACCCACACGTCCGCGAGGTAACCACGACGACTACACAGCGCCGGCGAGGAAATTCTCCGCGTCCCGAGATTCTTGACTCATCCCGAGGGTACGGACGGATTCGCCGTGCCTAGAGGATTCCCCTTGGAACTGCGCCGAGGACGGGATGTCGGCACGGCAGGGATCCCCCAGCGGCGGCCATTTTCTTCTCTCTCTCGGTTTCTGTCCATGGTGGGATGCAGGGCGGTATTACGGCACCTTGCTCATATATAGCCCAGGGGTTCCATGGCCCCGAGAAGCGCGCGAATCTGTAGCACCCGATCATCGGCGGCAGTCTCGAGGGGCCCGTTG

También se ha extraído información de transcriptomas de maíz (Zea mays) de cantatadb (http://cantata.amu.edu.pl/DOWNLOADS/Zea_mays_lncrnas.fasta). Esta base de datos sólo posee RNA no codificantes largos.
Al igual que en ensembl, esta información también está en formato fasta, pero sin comprimir.
Realizaré el mismo ejercicio de extraer los primeros transcriptomas de esta base.

In [6]:
#método para leer los primeros num transcriptomas de un archivo fasta en cantatadb
def leer_transcriptomas_cantatadb(archivo, num):
    transcriptoma = ""
    f = open(archivo, "r")
    for linea in f:
        if linea.startswith(">"):
            num -= 1
            if transcriptoma != "":
                print("TRANSCRIPTOMA: " + transcriptoma + "\n")
                transcriptoma = ""
            if num < 0:
                return
            print("CABECERA: " + linea)
        else:
            transcriptoma += linea.rstrip("\n")

Y ahora leeré la información del archivo descargado.

In [7]:
#Leyendo los primeros transcriptomas del archivo de RNA no codificantes
print("INICIANDO...\n")
leer_transcriptomas_cantatadb(".\\httpcantata.amu.edu.pldownload.php\\Zea mays (maíz)\\Zea_mays_lncrnas.fasta", 3)
print("FIN...\n")

INICIANDO...

CABECERA: >CNT20212011

TRANSCRIPTOMA: TTGCCATGGTCACTAAATACAGTCGGTATCATCCAGATGCGTTAGATCCAAACCTAATTTTGCCTGTTCCATTATCCCACATGCTCCCTAATCACGACAATAATCAAGCAGAGCAGATAGAAATCAACAATTTGCAACCAGTTCAAGGCAATCCACAGTTACAGATGAAACATTAACGTATCTAGGATGCTCAACATAAAATTAGACCACATAATCGTCGAATATTCAAGTTCACACAACTGAAGCACAAATGCTTAAACCATCTCTAAGAGCAGTAGTAGGCATCACAAGCCACAGCAAAACAGAGCATTATGCCTCGGGTGTTGGCTGTGTCAGGCCGGACTTTGCAAGCAGCTTCCCCAGCGCTTCAGGAGAGCACACCTGGTACTGTACCTCCCCGGAGCCCGGCTGCAGAAACACCTCGGCCAGCTCCAGCTTCTCGGCAGTCAGGCTGGTGGAGTCCATGGTCTTGCTGAGGACCTTAAGCGCGAGGGCGACGGCCTCCTCGCGGGTCAAGCCGTCCTTGTAGTCCTGCTTGAGCATGGACTGCGCGGCCTGGCTGTTGGCCCCAACAGCGGCGGCCTTCCACCCGCCGTAGTTCCCGGATGGGTCGCTCATGTAGAGCTGGAAGCCGTGGTGCTTGTCCCACCCGGCGAATAGGAAGGAGACCCCGAAGGGGCGGAGGCCTCCGAACTGGGTGTATCCCTGCTTGGTGTCGCAGAGGGACTGGACGAGCTGCTCGACGGGGATGGGTTCCTGGTAGGAGAGCGCGTAGCGTTGGGCGTGGAGGCGGGCGGTGTTGATGAGGATGTTGGCGTCGGACATGATCCCGGCCACGGCGCACGCCAGGTGGGAGTCGATCTTGTACATCTTCTCGGCGGAGCGCGAGGTCTGGAGGAGCTTGGAGGGCTGACCCGGCGTTGCCGATCGCCTCCATCGCGTACTCC

A diferencia de ensembl, en la cabecera ya no tenemos la información de qué tipo de lncrna estamos procesando.